### Importing required libraries :

In [427]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
#from random import normal
import math
from sklearn.metrics import accuracy_score

### Creating gaussian distributions : 

In [428]:
def createdata(v):
    
    np.random.seed(0)
    dis1=np.random.normal(0,v,50)
    dis2=np.random.normal(5,v,50)
    dis3=np.random.normal(10,v,50)
    A=np.concatenate([dis1,dis2])
    X=np.concatenate([A,dis3])
    X
    return X


In [429]:
X=createdata(1)
X.shape

(150,)

### Plotting the data:

In [430]:
'''
def plot_hist(data):
    plt.figure(figsize=(15,7))
    for x in data:
        plt.hist(x, bins = 80, normed = True, alpha = 0.7)
    #plt.xlim(-10, 20)
    plt.figure(figsize=(15,7))
    
plot_hist(X)
'''

'\ndef plot_hist(data):\n    plt.figure(figsize=(15,7))\n    for x in data:\n        plt.hist(x, bins = 80, normed = True, alpha = 0.7)\n    #plt.xlim(-10, 20)\n    plt.figure(figsize=(15,7))\n    \nplot_hist(X)\n'

### Function to initialize random means, variances and prior probabilities: 

In [431]:
def initialize(X,K):
    
    m=np.array([2.,3.,7.])             #random means
    #m=np.random.choice(X,size=3)      
    v=np.ones((3))                     #random variances set to 1 initially
    w=np.ones((K))/K                   #random prior weights set to equal
    #posterior probability that each input belongs to each of the 3 gaussians
    posterior=np.zeros((len(X),K),dtype=float)
    return m,v,w,posterior

In [432]:
means,var,wprior,posterior=initialize(X,K)
means,var,wprior,posterior.shape

(array([2., 3., 7.]),
 array([1., 1., 1.]),
 array([0.33333333, 0.33333333, 0.33333333]),
 (150, 3))

### True assignment of data samples to respective Gaussians and means:

In [475]:
Y=[0]*50 + [1]*50 + [2]*50
Y
M=[0,5,10]

### Function to allocate the probability of each of the data points belonging to each one of the clusters(Soft Allocation):

In [476]:
M

[0, 5, 10]

In [507]:
def GMM(X,K,epochs):
    N=len(X)
    #K=3
    pi=math.pi
    iterations=epochs
    M=np.array([0,5,10])
    
    for iter in range(iterations):
        
        #Expection Step : Calculating Posterior probabilities for 
        for i in range(len(X)):
            for k in range(K):

                part1=1/math.sqrt(2*pi*(var[k]))
                part2=(-1/2)*((X[i]-means[k])**2)
                part3= part2/(var[k])
                part4=np.exp(part3)
                posterior[i][k]=part1 * part4

                #posterior[i][k]=wprior[k]*(1/math.sqrt(2*var[k]*pi))*(math.exp(-(1/2)*((X[i]-means[k])**2)))

            posterior[i]=posterior[i]/np.sum(posterior[i])
        #print(posterior[0:2])
        
        #Maximization Step : Updating parameters : mean, variance, and prior probabilities :
        for i in range(K):
            wprior[i]=np.sum(posterior[:,i])/N
            means[i]=np.dot(posterior[:,i],X[:].T)/(np.sum(posterior[:,i]))
            var[i]=np.dot(posterior[:,i],((X-means[i])**2).T)/(np.sum(posterior[:,i]))
        
        
        li=[]
        for i in range(len(X)):
            li.append(np.argmax(posterior[i]))
        print("Epoch " + str(iter) + " : " + "priorprob="+ str(wprior))
        print("Epoch " + str(iter) + " : " + "mean="+ str(means))
        print("Epoch " + str(iter) + " : "+ "var=" + str(var))
        
        
        
        #accumean=accuracy_score(means,M)
        #print("Accuracy of means after Epoch {} : {}%".format(iter+1, accumean))
        accuracy = round((accuracy_score(li, li1)*100),2)
        print("Accuracy of allocation after Epoch {} : {}%".format(iter+1, accuracy))
        accuracy
        '''
        diff=[]
        for m in range(len(means)):
            diff[m]=(means[m].T-M[m].T)
        print(diff)    
        '''
        print(means.shape, M.shape)    
            

    return (wprior, means,var,accuracy,li)
    #means.shape


In [508]:
wprior,means,var,accuracy,li=GMM(X,3,4)

Epoch 0 : priorprob=[0.34189327 0.34202421 0.31608253]
Epoch 0 : mean=[ 2.28926901  3.17016952 10.30877374]
Epoch 0 : var=[7.40977473 7.84458953 1.03734907]
Accuracy of allocation after Epoch 1 : 70.67%
(3,) (3,)
Epoch 1 : priorprob=[0.34189534 0.34203174 0.31607292]
Epoch 1 : mean=[ 2.28310396  3.17652176 10.30879134]
Epoch 1 : var=[7.4023968  7.84204846 1.03733464]
Accuracy of allocation after Epoch 2 : 70.67%
(3,) (3,)
Epoch 2 : priorprob=[0.34189743 0.34203956 0.31606301]
Epoch 2 : mean=[ 2.27678737  3.18303087 10.30880955]
Epoch 2 : var=[7.3947869  7.83933851 1.03731973]
Accuracy of allocation after Epoch 3 : 70.67%
(3,) (3,)
Epoch 3 : priorprob=[0.34189952 0.34204768 0.3160528 ]
Epoch 3 : mean=[ 2.27031251  3.18970378 10.3088284 ]
Epoch 3 : var=[7.38693325 7.83644832 1.03730434]
Accuracy of allocation after Epoch 4 : 70.67%
(3,) (3,)


In [462]:
posterior

array([[5.30488337e-01, 4.69511663e-01, 9.38748785e-14],
       [5.48374064e-01, 4.51625936e-01, 3.38982576e-18],
       [5.41607348e-01, 4.58392652e-01, 3.05413108e-16],
       [5.22646020e-01, 4.77353980e-01, 2.45478529e-12],
       [5.28856150e-01, 4.71143850e-01, 1.93288952e-13],
       [5.59657211e-01, 4.40342789e-01, 2.88766621e-23],
       [5.41970811e-01, 4.58029189e-01, 2.45770702e-16],
       [5.53704763e-01, 4.46295237e-01, 3.72096971e-20],
       [5.53282849e-01, 4.46717151e-01, 5.56457836e-20],
       [5.48262623e-01, 4.51737377e-01, 3.68441950e-18],
       [5.50985071e-01, 4.49014929e-01, 4.28441857e-19],
       [5.35141116e-01, 4.64858884e-01, 1.03281226e-14],
       [5.44294949e-01, 4.55705051e-01, 5.77473590e-17],
       [5.51201943e-01, 4.48798057e-01, 3.56841360e-19],
       [5.47904974e-01, 4.52095026e-01, 4.80227496e-18],
       [5.49074458e-01, 4.50925542e-01, 1.99041710e-18],
       [5.34562725e-01, 4.65437275e-01, 1.37673453e-14],
       [5.54166510e-01, 4.45833

###  Training Accuracy after last epoch :

In [463]:
accuracy = round((accuracy_score(li, li1)*100),2)
print("Accuracy of allocation after Epoch {} : {}%".format(iter+1, accuracy))
print("Number of samples classifed for Gaussians 0, 1, 2 : ",li.count(0),li.count(1),li.count(2))


Accuracy of allocation after Epoch 4 : 72.0%
Number of samples classifed for Gaussians 0, 1, 2 :  52 48 50


* Initially each of the Gaussians consisted of 50 data points. Here we can see that it successfully predicted and classified the total 150 data points in 3 respective Gaussians(0,1,2). 
* Here I ran my algortithm for 4 epochs since I noticed that after 4th epoch my algorithm started to show no change in parameters which are mean, variance and prior probabilities. Also, when I how many data points were part of each cluster, a

### Trying different variances :

### Configuration 2:

In [454]:
X1=createdata(3)
means,var,wprior,posterior=initialize(X1,K)
wprior,means,var,accuracy2,li2=GMM(X1,3,10)

Epoch 0 : priorprob=[0.30379591 0.16129193 0.53491216]
Epoch 0 : mean=[-0.41752964  3.32022977  9.28651244]
Epoch 0 : var=[ 7.26556547  1.90157733 10.5107976 ]
Training Accuracy after Epoch 1 : 76.0%
Epoch 1 : priorprob=[0.27202643 0.24936382 0.47860975]
Epoch 1 : mean=[-0.52296372  3.70009393  9.60227255]
Epoch 1 : var=[ 8.99009478  2.86158074 11.11711744]
Training Accuracy after Epoch 2 : 77.33%
Epoch 2 : priorprob=[0.27711229 0.28848778 0.43439994]
Epoch 2 : mean=[-0.38087645  4.07237943  9.9145151 ]
Epoch 2 : var=[ 9.60834796  3.74225517 11.1201174 ]
Training Accuracy after Epoch 3 : 85.33%
Epoch 3 : priorprob=[0.28845632 0.31073309 0.40081059]
Epoch 3 : mean=[-0.22397572  4.39244573 10.16909213]
Epoch 3 : var=[ 9.91928415  4.37978983 11.02254412]
Training Accuracy after Epoch 4 : 89.33%
Epoch 4 : priorprob=[0.30032895 0.32254957 0.37712148]
Epoch 4 : mean=[-0.08931848  4.64172266 10.35684882]
Epoch 4 : var=[10.10580457  4.7823485  10.91583297]
Training Accuracy after Epoch 5 : 92.

### Configuration 3 :

In [455]:
X2=createdata(5)
means,var,wprior,posterior=initialize(X2,K)
wprior,means,var,accuracy3,li3=GMM(X2,3,10)

Epoch 0 : priorprob=[0.32099957 0.14138499 0.53761544]
Epoch 0 : mean=[-1.54752871  3.31178119 10.51948588]
Epoch 0 : var=[13.98669165  1.8805     18.07563298]
Training Accuracy after Epoch 1 : 72.0%
Epoch 1 : priorprob=[0.28882951 0.22971814 0.48145235]
Epoch 1 : mean=[-1.54117813  3.5851093  10.90137209]
Epoch 1 : var=[18.62759833  3.00261374 19.44275524]
Training Accuracy after Epoch 2 : 76.67%
Epoch 2 : priorprob=[0.29346914 0.26741311 0.43911776]
Epoch 2 : mean=[-1.24711314  3.90304076 11.27074235]
Epoch 2 : var=[21.06948723  4.31769766 19.66699698]
Training Accuracy after Epoch 3 : 81.33%
Epoch 3 : priorprob=[0.30189226 0.29336717 0.40474057]
Epoch 3 : mean=[-0.98427124  4.18930851 11.6001626 ]
Epoch 3 : var=[22.55781724  5.51393345 19.6492959 ]
Training Accuracy after Epoch 4 : 82.0%
Epoch 4 : priorprob=[0.31056034 0.30897623 0.38046343]
Epoch 4 : mean=[-0.77434744  4.40934265 11.84086901]
Epoch 4 : var=[23.46444577  6.49525926 19.59273688]
Training Accuracy after Epoch 5 : 86.0

### Configuration 4 : 

In [456]:
X2=createdata(10)
means,var,wprior,posterior=initialize(X2,K)
wprior,means,var,accuracy4,li4=GMM(X2,3,10)

Epoch 0 : priorprob=[0.38434615 0.08659788 0.52905597]
Epoch 0 : mean=[-4.6580071   3.11694109 14.69448081]
Epoch 0 : var=[37.54829617  2.02034978 48.59829648]
Training Accuracy after Epoch 1 : 56.67%
Epoch 1 : priorprob=[0.3477011  0.17256861 0.47973029]
Epoch 1 : mean=[-4.67859324  3.1957804  15.27753612]
Epoch 1 : var=[48.64916174  3.46394392 51.86050917]
Training Accuracy after Epoch 2 : 59.33%
Epoch 2 : priorprob=[0.34473224 0.2027475  0.45252027]
Epoch 2 : mean=[-4.41569306  3.30310175 15.703987  ]
Epoch 2 : var=[54.56295141  5.66824926 51.36808334]
Training Accuracy after Epoch 3 : 61.33%
Epoch 3 : priorprob=[0.33710872 0.23663704 0.42625424]
Epoch 3 : mean=[-4.297512    3.41666938 16.17357413]
Epoch 3 : var=[58.94737614  8.88535254 49.90441664]
Training Accuracy after Epoch 4 : 62.0%
Epoch 4 : priorprob=[0.32721513 0.26927694 0.40350793]
Epoch 4 : mean=[-4.23556038  3.4781826  16.61226677]
Epoch 4 : var=[62.48603288 12.46709236 48.20153823]
Training Accuracy after Epoch 5 : 62.

* We can see a decrease in accuracies when we ran our algorithm for less number of epochs. In our algorithm, we stopped when our posterior probabilities showed almost no change. After trying different values for variance, we noticed that as variance increases, our GMM algorithm takes many iterations to converge. GMM is much more flexible allowing us to generate much better fitting distributions.